In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/'My Drive'/personal_projects/ta-lib

/content/drive/My Drive/personal_projects/ta-lib


In [3]:
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 144618 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 4.4MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437782 sha256=7393631016c55403c931cb7fe505dd987d921b63d0e2ea0ead566493eda9fabd
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [4]:
%cd ..

/content/drive/My Drive/personal_projects


In [5]:
!nvidia-smi

Mon Oct  5 13:12:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# common imports

import numpy as np
import os
import pickle
import random
import numpy as np
import math
from talib import NATR, RSI

In [7]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Masking, Concatenate, Conv2D, MaxPooling2D, Flatten
from keras.layers.core import Activation, Dropout, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from hyperas import optim
# from hyperas.distributions import choice, uniform
# from hyperopt import Trials, STATUS_OK, tpe
from keras import optimizers
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [8]:
# model_interface.py

# This is an abstract class. You need to implement yours.
class AbstractModelBuilder(object):

  def __init__(self, weights_path = None):
    self.weights_path = weights_path
    self.model = None

  def loadModel(self):
    weights_path = self.weights_path
    self.model = self.buildModel()

    if weights_path and os.path.isfile(weights_path):
      try:
        self.model.load_weights(weights_path)
      except e:
        print(e)

  # You need to override this method.
  def buildModel(self):
    raise NotImplementedError("You need to implement your own model.")

  def predict(self, data):
    raise NotImplementedError("You need to implement predictor function")

  def compile_model(self):
    raise NotImplementedError("You need to implement this function")

  def trainModel(self, inputs, targets):
    raise NotImplementedError("You need to implement this function")
  
  def printSummary(self):
    raise NotImplementedError("You need to implement this function")

In [9]:

# do a cut off
# unable to build (PnL transfer t to t+1)?
# Doesn't work
# class lstm_model(AbstractModelBuilder):
  
#   def buildModel(self):
#       inputs = []
    
#       weekly_inputs = Input(shape=(None, num_encoder_tokens))
#       inputs.append(weekly_inputs)
    
#       encoder_inputs = Dense(8, activation='relu')(weekly_inputs)
#       week_encoder = LSTM(16, return_state=True)
#       encoder_outputs, state_h, state_c = week_encoder(encoder_inputs)
#       encoder_states = [state_h, state_c]

#       input_price_info = Input(shape=(None, 3))
#       inputs.append(input_price_info)
#       inputs_rsi_macd_schaff = Input(shape=(None, 4))
#       inputs.append(inputs_rsi_macd_schaff)
#       input_bolinger = Input(shape=(None, 3))
#       inputs.append(input_bolinger)
#       input_adx = Input(shape(None, 3))
#       inputs.append(input_adx)
#       input_one_hot = Input(shape(None, 8))
#       inputs.append(input_one_hot)

#       x1 = Dense(6, activation='leaky relu')(input_price_info)
#       x2 = Dense(6, activation='leaky relu')(input_adx)
#       x3 = Dense(6, activation='leaky relu')(input_bolinger)
#       x4 = Dense(8, activation='leaky relu')(input_one_hot)

#       feature_vector = Concatenate(axis=1)(x1, inputs_rsi_macd_schaff, x2, x3, x4)
#       x = Dense(64, activation='relu')(feature_vector)
#       embedding = Dropout(0.5)(x)

#       layer_1 = LSTM(64, return_sequences=True)
#       x = layer_1(embedding, initial_state=encoder_states)
#       output_layer_1 = Activation('leaky relu')(x)

#       layer_2 = LSTM(32, return_sequences=True)
#       x = layer_2(output_layer_1)
#       output_layer_2 = Activation('leaky relu')(x)

#       final_dense = Dense(3, activation='linear')
#       final_output = final_dense(output_layer_2)

#       model = Model(inputs, final_output)
#       return model

# Take past 30 timesteps one hot veectors then use cnn and give it to the neural net 


class cnn_model(AbstractModelBuilder):

  def buildModel(self):
    hourly_data = Input(shape=(1, 30, 3))

    x = Conv2D(2, (1,3), activation='relu')(hourly_data)
    x = MaxPooling2D(pool_size=(1,2))(x)
    x = Conv2D(4, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2))(x)
    x = Conv2D(4, (1,3), activation='relu')(x)
    x = Flatten()(x)
    x0 = Dense(8, activation='relu')(x)

    input_PnL = Input(shape=(9,))
    x1 = Dense(4)(input_PnL)
    x1 = LeakyReLU()(x1)

    # scaling
    input_scale = Input(shape=(30,))
    x = Dense(4, activation='relu')(input_scale)
    x_scale = Dense(1)(x)

    inputs = [hourly_data, input_PnL, input_scale]
    input_price_info = Input(shape=(1, 60, 2))
    inputs.append(input_price_info)
    input_indicators = Input(shape=(1, 60, 5))
    inputs.append(input_indicators)
    input_bolinger = Input(shape=(4, 60, 1))
    inputs.append(input_bolinger)
    input_adx = Input(shape=(1, 60, 3))
    inputs.append(input_adx)
    input_nb = Input(shape=(1, 30, 2))
    inputs.append(input_nb)
    input_onehot = Input(shape=(10, 30, 1))
    inputs.append(input_onehot)

    # price
    x = Conv2D(5, (1,3), activation='relu')(input_price_info)
    x = Conv2D(8, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(12, (1,3), activation='relu')(x)
    x = Conv2D(14, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(16, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Flatten()(x)
    x = Dense(25, activation='relu')(x)
    x_price = Dropout(0.3)(x)

    # nifty bank
    x = Conv2D(3, (1,3), activation='relu')(input_nb)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(4, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(5, (1,3), activation='relu')(x)
    x = Flatten()(x)
    x_nb = Dense(6)(x)

    # adx
    x = Dense(1)(input_adx)
    x = LeakyReLU()(x)
    indicators = Concatenate(axis=3)([input_indicators, x, input_adx[:,:,:,1:]])

    # bolinger
    x = Conv2D(3, (4,3), activation='relu')(input_bolinger)
    x = Conv2D(4, (1,5), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(5, (1,7), activation='relu')(x)
    x = Conv2D(6, (1,10), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Flatten()(x)
    x = Dense(10, activation='relu')(x)
    x_bolinger = Dropout(0.3)(x)

    # onehot
    x = Conv2D(8, (8,3), activation='relu')(input_onehot)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(10, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(12, (1,5), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(10, 'relu')(x)
    x_onehot = Dropout(0.2)(x)

    #indicators
    x = Conv2D(8, (1,3), activation='relu')(indicators)
    x = Conv2D(12, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x) 
    x = Conv2D(12, (1,3), activation='relu')(x)
    x = Conv2D(16, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Conv2D(20, (1,3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(1,2), strides=(1,2))(x)
    x = Flatten()(x)
    x = Dense(20, activation='relu')(x)
    x_indicators = Dropout(0.3)(x)

    # concatenate and combine all the feature_vectors
    x = Concatenate(axis=1)([x_price, x_bolinger, x_indicators, x_onehot, x_nb])
    x = Dense(30)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.5)(x)

    # concatenate context
    x = Concatenate(axis=1)([x, x1, x0])
    x = Dense(23)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.5)(x)

    x = Dense(12)(x)
    x = LeakyReLU()(x)

    # Experiment with tanh too
    x = Dense(3, activation='linear')(x)

    # provide scaling
    # possible to force x_scale > 0 ? 
    final_output = x * x_scale

    # num trainable params large ~ 20k ? 
    model = Model(inputs, final_output)

    return model


  def compile_model(self):
    self.model.compile(loss='mse', optimizer='adam')

  def predict(self, data, sub_sample=None):
    if sub_sample is None:
      return self.model.predict([data['hourly_data'],   
                      data['portfolio'],
                      data['scale'], 
                      data['price_curve'],
                      data['indicators'],
                      data['bolinger'],
                      data['adx'],
                      data['nb'],
                      data['onehot']])
    else:
      return self.model.predict([data['hourly_data'][sub_sample],   
                      data['portfolio'][sub_sample],
                      data['scale'][sub_sample], 
                      data['price_curve'][sub_sample],
                      data['indicators'][sub_sample],
                      data['bolinger'][sub_sample],
                      data['adx'][sub_sample],
                      data['nb'][sub_sample],
                      data['onehot'][sub_sample]])


  def batchTrain(self, data, targets):
    return self.model.train_on_batch([data['hourly_data'],    
                      data['portfolio'],
                      data['scale'], 
                      data['price_curve'],
                      data['indicators'],
                      data['bolinger'],
                      data['adx'],
                      data['nb'],
                      data['onehot']],
                      targets)
  
  def fitModel(self, data, targets, batch_size, epoch):
    return self.model.fit([data['hourly_data'],    
                      data['portfolio'],
                      data['scale'], 
                      data['price_curve'],
                      data['indicators'],
                      data['bolinger'],
                      data['adx'],
                      data['nb'],
                      data['onehot']],
                      targets,
                      batch_size=batch_size,
                      epochs=epoch)
  
  def printSummary(self):
    print(self.model.summary())

In [40]:
# Improvements
# prioritized Experience Replay
# Dual Target Network
# Better Optimzation techique
class ExperienceReplay(object):
  
  def __init__(self, max_memory=200000, discount=.9):
    self.max_memory = max_memory
    self.count_range = {}
    self.memory = list()
    self.discount = discount
    self.record_sort = False

  def remember(self, states, game_over):
    for i in range(p_games):
      save_states_tm1, save_states_t = {}, {}
      for key in states[0].keys():
        save_states_tm1[key] = states[0][key][i]
        save_states_t[key] = states[3][key][i]
      self.memory.append([[save_states_tm1, int(states[1][i]), states[2][i][0], save_states_t], game_over, 0])
    if self.record_sort:
      if self.count_range.__contains__(0):
        l, r = self.count_range[0]
        self.count_range[0] = (l, r + len(states[0]))
      else:
        self.count_range[0] = (len(self.memory) - 1, len(self.memory) - 1 + len(states[0]))
    
    if len(self.memory) > self.max_memory:
      del self.memory[:30000]

  # keep record sorted
  def _adjust_record(self, idx):
    if not self.record_sort:
      return

    old_count = self.memory[idx][2]
    self.memory[idx][2] = old_count + 1
    old_l,old_r = self.count_range[old_count]
  
    if idx != old_l:
      temp = self.memory[idx]
      self.memory[idx] = self.memory[old_l]
      self.memory[old_l] = temp

    if self.count_range.__contains__(old_count + 1):
      new_l,new_r = self.count_range[old_count + 1]
      self.count_range[old_count + 1] = (new_l, new_r + 1)
    else:
      self.count_range[old_count + 1] = (old_l, old_l)

    if old_l + 1 > old_r:
      del self.count_range[old_count - 1]
    else:
      self.count_range[old_count - 1] = (old_l + 1, old_r)

  # use it for fast speed (uses a lot more memory)
  def get_offline_batch(self, env, model_interface, batch_size):
    len_memory = len(self.memory)
    size = min(len_memory, batch_size)
    num_actions = model_interface.model.output_shape[-1]
    inputs = {}
    inputs_t = {}
    reward = np.empty(size)
    over = np.empty(size)
    action = np.zeros((size, num_actions), dtype=bool)
    price_curve, indicators, bolinger, adx, nb, onehot, portfolio, hourly_data, scale_data = self.allocate(size)
    price_curve_t, indicators_t, bolinger_t, adx_t, nb_t, onehot_t, portfolio_t, hourly_data_t, scale_data_t = self.allocate(size)

    for i, idx in enumerate(np.random.randint(0, len_memory, size=size)):
      state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
      
      price_curve[i] = state_t['price_curve']
      indicators[i] = state_t['indicators']
      bolinger[i] = state_t['bolinger']
      adx[i] = state_t['adx']
      onehot[i] = state_t['onehot']
      nb[i] = state_t['nb']
      portfolio[i] = state_t['portfolio']
      hourly_data[i] = state_t['hourly_data']
      scale_data[i] = state_t['scale']
      
      price_curve_t[i] = state_tp1['price_curve']
      indicators_t[i] = state_tp1['indicators']
      bolinger_t[i] = state_tp1['bolinger']
      adx_t[i] = state_tp1['adx']
      onehot_t[i] = state_tp1['onehot']
      nb_t[i] = state_tp1['nb']
      portfolio_t[i] = state_tp1['portfolio']
      hourly_data_t[i] = state_tp1['hourly_data']
      scale_data_t[i] = state_tp1['scale']

      reward[i] = reward_t
      action[i][action_t] = True
      over[i] = float(self.memory[idx][1])
    
    inputs['price_curve'] = price_curve
    inputs['indicators'] = indicators
    inputs['bolinger'] = bolinger
    inputs['adx'] = adx
    inputs['onehot'] = onehot
    inputs['portfolio'] = portfolio
    inputs['nb'] = nb
    inputs['scale'] = scale_data
    inputs['hourly_data'] = hourly_data

    inputs_t['price_curve'] = price_curve_t
    inputs_t['indicators'] = indicators_t
    inputs_t['bolinger'] = bolinger_t
    inputs_t['adx'] = adx_t
    inputs_t['onehot'] = onehot_t
    inputs_t['portfolio'] = portfolio_t
    inputs_t['nb'] = nb_t
    inputs_t['scale'] = scale_data_t
    inputs_t['hourly_data'] = hourly_data_t

    targets = model_interface.predict(inputs)
    targets[action] = reward + self.discount * over * np.max(model_interface.predict(inputs_t), axis=1)

    return inputs, targets

  def allocate(self, size):
    p_s = list(self.memory[0][0][0]['price_curve'].shape)
    i_s = list(self.memory[0][0][0]['indicators'].shape)
    b_s = list(self.memory[0][0][0]['bolinger'].shape)
    a_s = list(self.memory[0][0][0]['adx'].shape)
    o_s = list(self.memory[0][0][0]['onehot'].shape)
    pf_s = list(self.memory[0][0][0]['portfolio'].shape)
    n_s = list(self.memory[0][0][0]['nb'].shape)
    s_s = list(self.memory[0][0][0]['scale'].shape)
    h_s = list(self.memory[0][0][0]['hourly_data'].shape) 

    return np.empty([size] + p_s), np.empty([size] + i_s), np.empty([size] + b_s), np.empty([size] + a_s), np.empty([size] + n_s), np.empty([size] + o_s), np.empty([size] + pf_s), np.empty([size] + h_s), np.empty([size] + s_s)

  # used for low memory usage
  def get_online_batch(self, env, model_interface, batch_size=64):
    len_memory = len(self.memory)
    num_actions = model_interface.model.output_shape[-1]
    inputs = {}
    price_curve, indicators, bolinger, adx, nb, onehot, portfolio, hourly_data, scale_data = [], [], [], [], [], [], [], [], []

    targets = np.zeros((min(len_memory, batch_size), num_actions))
    for i, idx in enumerate(np.random.randint(0, len_memory, size=min(len_memory, batch_size))):
      pnl_state_t, action_t, reward_t, pnl_state_tp1, code, day, index = self.memory[idx][0]
      self._adjust_record(idx)
      state_t = env.generate_data(code, day, index)
      state_t['portfolio'] = pnl_state_t
      game_over = self.memory[idx][1]
      targets[i] = model_interface.predict(state_t)[0]
      if not game_over:
        state_tp1 = env.generate_data(code, day, index + 1)
        state_tp1['portfolio'] = pnl_state_tp1
        Q_sa = np.max(model_interface.predict(state_tp1)[0])
        targets[i, action_t] = reward_t + self.discount * Q_sa
      else:
        targets[i, action_t] = reward_t

      price_curve.append(np.squeeze(state_t['price_curve'], axis=0))
      indicators.append(np.squeeze(state_t['indicators'], axis=0))
      bolinger.append(np.squeeze(state_t['bolinger'], axis=0))
      adx.append(np.squeeze(state_t['adx'], axis=0))
      onehot.append(np.squeeze(state_t['onehot'], axis=0))
      nb.append(np.squeeze(state_t['nb'], axis=0))
      portfolio.append(np.squeeze(state_t['portfolio'], axis=0))
      hourly_data.append(np.squeeze(state_t['hourly_data'], axis=0))
      scale_data.append(np.squeeze(state_t['scale'], axis=0))


    inputs['price_curve'] = np.array(price_curve)
    inputs['indicators'] = np.array(indicators)
    inputs['bolinger'] = np.array(bolinger)
    inputs['adx'] = np.array(adx)
    inputs['onehot'] = np.array(onehot)
    inputs['portfolio'] = np.array(portfolio)
    inputs['nb'] = np.array(nb)
    inputs['scale'] = np.array(scale_data)
    inputs['hourly_data'] = np.array(hourly_data)

    return inputs, targets

In [11]:
path_list = ['BAJFINANCE/processed_data', 'AXISBANK/processed_data', 'HDFC/processed_data', 'HDFCBANK/processed_data', 'ICICIBANK/processed_data']
X_price_info, X_indicators, X_adx, X_bolinger, X_onehot, X_nb = [{} for _ in range(6)]
X_avg_val, X_high_val, X_low_val, X_close_val = [{} for _ in range(4)]

code = 0
for path in path_list:
  if not os.path.exists(path):
    print('path not found', path)
  else:
    f = open(path, 'rb')
    obj = pickle.load(f)
    X_price_info[code] = obj['X_price_info']
    X_indicators[code] = obj['X_indicators']
    X_adx[code] = obj['X_adx']
    X_bolinger[code] = obj['X_bolinger']
    X_onehot[code] = obj['X_onehot']
    X_nb[code] = obj['X_nb']
    X_avg_val[code] = obj['avg_val']
    X_high_val[code] = obj['high_val']
    X_low_val[code] = obj['low_val']
    X_close_val[code] = obj['close_val']
    code += 1
    f.close()

In [12]:
# from keras.optimizers import SGD
modelFilename=''
model_interface = cnn_model(modelFilename)
model_interface.loadModel()
model_interface.compile_model()

# sgd = SGD(lr = 0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
# need a better optimization/random optimization techique.
# maybe try Particle swarm or genectic algorithm

In [41]:
# Initialize experience replay object
exp_replay = ExperienceReplay()

In [34]:
temp = exp_replay.memory

In [36]:
exp_replay.memory = temp

In [45]:
len(exp_replay.memory)

85800

In [38]:
np.argmax([1,2,4])

2

In [ ]:
model_interface.predict(input_t)

array([[ 1.4604977e-04,  2.3279474e-05, -7.8652592e-05]], dtype=float32)

In [ ]:
model_interface.model.save_weights("model_checkpoint_day_155.h5" if modelFilename == None else modelFilename, overwrite=True)

In [19]:
scope = 60
p_games = 100
env = MarketEnv(X_price_info,
        X_indicators,
        X_bolinger,
        X_adx,
        X_onehot,
        X_nb,
        X_avg_val,
        X_close_val,
        X_high_val,
        X_low_val,
        scope,
        parallel_games=p_games)

In [42]:
# parameters
epsilon = 0.5  # exploration
large_batches = True  # train in large batches for less I/O overhead.
min_epsilon = 0.01
batch_size = 16384 if large_batches else 64
discount = 0.9    # increasing future discount incentivise future P
epoch = 3
loss = 0.
stop = False
save_cycle = 3 # epoch // 25
action = np.zeros(p_games)

for e in range(epoch):
  input_t = env.reset()
  day_end = False
  cumReward, realized_pnl, per_success, num_trades = 0., 0., 0, 0
  
  while not day_end:
    input_tm1 = input_t
    isRandom = np.random.rand(p_games) <= epsilon
    action[isRandom] = np.random.randint(0, 3, size=np.sum(isRandom))
    if len(isRandom) < p_games:
      prediction = model_interface.predict(input_tm1, np.logical_not(isRandom))
      if np.nan in prediction:
        print("OCCUR NaN!!!")
        stop = True
        break
      action[np.logical_not(isRandom)] = np.argmax(prediction, axis=1)
    input_t, reward, pnl, success, day_end = env.step(action)
    exp_replay.remember([input_tm1, action, reward, input_t], day_end)
    cumReward += np.sum(reward)
    realized_pnl += np.sum(pnl)
    per_success += np.sum(success[success > 0])
    num_trades += np.sum(np.abs(success))

  
  print('epoch =', str(e), 'cumReward =', str(cumReward / p_games), 'pnl =', str(realized_pnl / p_games), 'percentage success', str(float(per_success) * 100 / max(num_trades, 1)))
  
  if stop:
    break
  
  # train
  if large_batches:
    inputs, targets = exp_replay.get_offline_batch(env, model_interface, batch_size)
    model_interface.fitModel(inputs, targets, 64, 10)
  else:
    inputs, targets = exp_replay.get_online_batch(env, model_interface)
    loss = model_interface.batchTrain(inputs, targets)
    print('batch = ', str(e), 'loss = ', str(loss))
  
  if not (e + 1) % save_cycle:
    epsilon = max(min_epsilon, epsilon * 0.95)
    model_interface.model.save_weights("model.h5", overwrite=True)

epoch = 0 cumReward = 0.052081829135809145 pnl = -0.007215079192740983 percentage success 48.71656309105647
Epoch 1/10
256/256 [==============================] - 2s 9ms/step - loss: 1.8200e-04
Epoch 2/10
256/256 [==============================] - 2s 9ms/step - loss: 1.0706e-04
Epoch 3/10
256/256 [==============================] - 2s 9ms/step - loss: 9.4139e-05
Epoch 4/10
256/256 [==============================] - 2s 9ms/step - loss: 8.9351e-05
Epoch 5/10
256/256 [==============================] - 2s 9ms/step - loss: 8.8702e-05
Epoch 6/10
256/256 [==============================] - 2s 9ms/step - loss: 8.4195e-05
Epoch 7/10
256/256 [==============================] - 2s 9ms/step - loss: 8.2106e-05
Epoch 8/10
256/256 [==============================] - 2s 9ms/step - loss: 8.1229e-05
Epoch 9/10
256/256 [==============================] - 2s 9ms/step - loss: 8.1335e-05
Epoch 10/10
256/256 [==============================] - 2s 9ms/step - loss: 8.1218e-05
epoch = 1 cumReward = 0.06366773873076789

In [44]:
env.position

array([[ 0.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 0.],
       [ 1.],
       [-1.],
       [ 0.],
       [-1.],
       [-1.],
       [-1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [-1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 0.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.],
      

In [18]:
class MarketEnv(object):

    ON_PROFIT_HOLDING = 0.9

    def __init__(self, x_price_info, x_indicators, x_bolinger, x_adx, x_onehot, x_nb, x_avg_val, x_close_val, x_high_val, x_low_val, scope, parallel_games=1):
        
        max_size = 0
        self.index_end = []
        self.num_target = X_price_info.__len__()
        for key in X_price_info.keys():
            self.index_end.append(len(X_price_info[key]))
            max_size = max(max_size, self.index_end[-1])
        
        self.X_price_info = self._copy_table(x_price_info, max_size)
        self.X_indicators = self._copy_table(x_indicators, max_size)
        self.X_bolinger = self._copy_table(x_bolinger, max_size)
        self.X_adx = self._copy_table(x_adx, max_size)
        self.X_onehot = self._copy_table(x_onehot, max_size)
        self.X_nb = self._copy_table(x_nb, max_size)
        
        self.X_avg_val = x_avg_val
        self.X_close_val = x_close_val
        self.X_high_val = x_high_val
        self.X_low_val = x_low_val
        self.scope = scope
        self.parallel_games = parallel_games
        self.done = False
        self.position, self.reward, self.boughts, self.budget, self.position, self.realized_pnl, self.success, self.num_position, self.cum_reward = [np.zeros((parallel_games, 1)) for _ in range(9)]
        self.num_days = self.X_price_info.shape[1]
        self.episode_length = self.X_price_info.shape[2]
        self.targetContext = np.zeros((self.num_target, self.num_days), dtype=bool)

        self.actions = [
            "LONG",
            "HOLD",
            "SHORT"
        ]

        self.ON_LOSS_HOLDING = np.array([1.5, 1.2, 1.0, 0.8, 0.3, 0.9, 2.0, 3.0, 4.0])

        # self.action_space = spaces.Discrete(len(self.actions))
        # self.observation_space = spaces.Box(np.ones(scope * (len(input_codes) + 1)) * -1, np.ones(scope * (len(input_codes) + 1)))

        # self.reset()
        self._seed()
    
    def _copy_table(self, x_array, max_size):
        X_array = np.empty([self.num_target, max_size] + list(x_array[0].shape[1:]))
        for i in range(self.num_target):
            diff = max_size - x_array[i].shape[0]
            if diff > 0:
                X_array[i] = np.concatenate([x_array[i], np.zeros([diff] + list(x_array[0].shape[1:]))], axis=0)
            else:
                X_array[i] = x_array[i]
        return X_array

    def _get_holding_mutiplier(self, cum_reward, diff):     
        return (diff < 0) * self.ON_LOSS_HOLDING[np.clip(((2.0 - cum_reward) / 0.5).astype(int), 0, 8)] + (diff >= 0) * self.ON_PROFIT_HOLDING

    def _create_hourly_price_till(self, index, timesteps, code):
        minutes = timesteps * 60
        cur = np.zeros(self.parallel_games, dtype=int)
        hourly_avg_price, hourly_close_price, hourly_high_price, hourly_low_price = [np.zeros((self.parallel_games, timesteps)) for _ in range(4)]

        for k in range(self.parallel_games):
            start = max((index[k] + 1) % 60, index[k] - minutes + 1)
            end = min(len(self.X_close_val[code[k]]), index[k] + 1)
            for i in np.arange(start, end, 60):
                hourly_avg_price[k][cur[k]] = np.average(self.X_avg_val[code[k]][i: i + 60])
                hourly_close_price[k][cur[k]] = self.X_close_val[code[k]][i + 59]
                hourly_high_price[k][cur[k]] = np.max(self.X_high_val[code[k]][i: i + 60])
                hourly_low_price[k][cur[k]] = np.min(self.X_low_val[code[k]][i: i + 60])
                cur[k] += 1
        
        return (hourly_avg_price, hourly_close_price, hourly_high_price, hourly_low_price, cur)

    def step(self, action):
        if self.done:
            return self.state, self.reward, self.realized_pnl, self.success, self.done

        vari = np.expand_dims(self.target_price[:, self.currentTargetIndex, 1] / 100, axis=1)
        self.budget = self.budget / (1 + vari)
        change = self.boughts * vari
        self.boughts += change
        self.cum_reward = self.boughts - self.num_position * self.position
        action = np.expand_dims(1 - action, axis=1)
        distribute = np.abs(np.sign(action + self.position))
        self.realized_pnl = (1 - distribute) * self.cum_reward
        self.reward = self.realized_pnl + distribute * self._get_holding_mutiplier(self.cum_reward, change) * change
        new_position = (self.budget >= 1) * action * distribute
        self.budget += (1 - distribute) * (self.num_position + self.cum_reward) - np.abs(new_position)
        self.position = np.sign(self.position * distribute + new_position)
        self.boughts = self.boughts * distribute + new_position
        self.num_position = np.abs(self.position * self.num_position + new_position)
        self.success = np.sign(self.realized_pnl)
        self.currentTargetIndex += 1

        # print('budget=',str(self.budget[0][0]),'bought=',str(self.boughts[0][0]),'position=',str(self.position[0][0]),'distribute=',str(distribute[0][0]),'new_position=',str(new_position[0][0]),'num_position=',str(self.num_position[0][0]),'cum_reward=',str(self.cum_reward[0][0]))

        if self.currentTargetIndex >= self.episode_length:
            self.done = True
        
        self._defineState()
        return self.state, self.reward, self.realized_pnl, self.success, self.done

    def reset(self):
        self.targetContext *= False
        x = np.array([np.random.rand() for _ in range(self.num_target)])
        x = (x / np.sum(x)) * self.parallel_games
        y = []
        for i in range(self.num_target - 1):
            y.append(int(x[i]))
        y.append(self.parallel_games - sum(y))
        y = np.random.permutation(y)
        self.targetDays = []
        self.targetCode = np.hstack([i * np.ones(y[i], dtype=int) for i in range(self.num_target)])
        for i in range(self.num_target):
            selection = np.sort(np.random.choice(range(8, self.index_end[i]), size=y[i], replace=False))
            self.targetContext[i][selection] = True
            self.targetDays = self.targetDays + list(selection)
        
        self.targetDays = np.array(self.targetDays)
        self.target_price = self.X_price_info[self.targetContext]
        self.target_indicators = self.X_indicators[self.targetContext]
        self.target_nb = self.X_nb[self.targetContext]
        self.target_bolinger = self.X_bolinger[self.targetContext]
        self.target_adx = self.X_adx[self.targetContext]
        self.target_onehot = self.X_onehot[self.targetContext]

        self.currentTargetIndex = self.scope - 1
        self.done = False
        self.budget = self.budget * 0. + 8. 
        self.position = self.position * 0.
        self.num_position = self.num_position * 0.
        self.boughts = self.boughts * 0.
        self.cum_reward = self.cum_reward * 0.
        self._defineState()
        return self.state

    def _render(self, mode='human', close=False):
        if close:
            return
        return self.state

    def _seed(self):
        return int(random.random() * 100)

    def _create_hourly_data(self, code, day, index):
        timesteps = 80
        lookback = self.scope // 2
        # adjusting day_ignored and index_ignored
        end = (day + 4) * 375 + (index + 30)
        hourly_avg_val, hourly_close_val, hourly_high_val, hourly_low_val, j = self._create_hourly_price_till(end, timesteps, code)
        hourly_rsi, hourly_atr = [], []
        X_avg_price = np.empty((self.parallel_games, 1, lookback, 1))
        for i in range(self.parallel_games):
          hourly_rsi.append((RSI(hourly_close_val[i][:j[i]], timeperiod=14)[-lookback:]) / 100)
          hourly_atr.append(NATR(hourly_high_val[i][:j[i]], hourly_low_val[i][:j[i]], hourly_close_val[i][:j[i]], timeperiod=14)[-lookback:])
          X_avg_price[i] = np.expand_dims((hourly_avg_val[i][j[i]-lookback:j[i]] - hourly_avg_val[i][j[i]-(lookback + 1) : j[i]-1]) * 10 / hourly_avg_val[i][j[i]-(lookback + 1) : j[i]-1], axis=(0, 2))
        hourly_rsi = np.expand_dims(np.array(hourly_rsi), axis=(1, 3))
        hourly_atr = np.expand_dims(np.array(hourly_atr), axis=(1, 3))
        hourly_data = np.concatenate([X_avg_price, hourly_rsi, hourly_atr], axis=3)

        return hourly_data

    def _standardize(self, X_array):
        n = X_array.shape[2]
        for i in range(n):
            temp = X_array[:, :, i]
            X_array[:, :, i] = (temp - np.mean(temp, axis=1, keepdims=True)) / np.std(temp, axis=1, keepdims=True)

    # to prevent memory overflow create training data in an online way
    def generate_data(self, code, day, index):
        r_data = {}

        if index < self.scope - 1:
            print('index not in range =', str(index))
            return None

        # normalize prices
        price_curve = np.array(self.target_price[:, index - self.scope + 1 : index + 1])
        r_data['scale'] = np.array(price_curve[:, self.scope // 2:, 1])
        self._standardize(price_curve)
        price_curve = np.expand_dims(price_curve, axis=1)

        # normalize nb prices
        nb_price = np.array(self.target_nb[:, index - (self.scope // 2) + 1 : index + 1])
        self._standardize(nb_price)
        nb_price = np.expand_dims(nb_price, axis=1)

        indicators = self.target_indicators[:, index - self.scope + 1 : index + 1]
        indicators = np.expand_dims(indicators, axis=1)
        
        bolinger = self.target_bolinger[:, index - self.scope + 1 : index + 1]
        bolinger = np.swapaxes(bolinger, 1, 2)
        bolinger = np.expand_dims(bolinger, axis=3)
        
        adx = self.target_adx[:, index - self.scope + 1 : index + 1]
        adx = np.expand_dims(adx, axis=1)
        
        onehot = self.target_onehot[:, index - (self.scope // 2) + 1 : index + 1]
        onehot = np.swapaxes(onehot, 1, 2)
        onehot = np.expand_dims(onehot, axis=3)

        r_data['price_curve'] = price_curve
        r_data['indicators'] = indicators
        r_data['bolinger'] = bolinger
        r_data['adx'] = adx
        r_data['onehot'] = onehot
        r_data['nb'] = nb_price
        r_data['hourly_data'] = self._create_hourly_data(code, day, index)

        return r_data

    def _defineState(self):
        if self.done:
            return
        
        tmpState = self.generate_data(self.targetCode, self.targetDays, self.currentTargetIndex)
        
        onehot_code = np.zeros((self.parallel_games, self.num_target), dtype=float)
        for i in range(self.parallel_games):
            onehot_code[self.targetCode[i]] = 1.0
        
        tmpState['portfolio'] = np.concatenate([self.cum_reward / 8, self.budget / 8, self.position, (float(self.currentTargetIndex) / self.episode_length) * np.ones((self.parallel_games, 1)), onehot_code], axis=1)
        self.state = tmpState







# import gym
# from gym import spaces

# idea don't penalize strictly on holding even when the price falls upto a certain level
# use the idea of stoploss i.e when the price fall below stoploss start penalizing heavily for subsequent drops
# create levels:

# rise :
# All values  :  MULTIPLIER = 0.9

# fall :
# [   > 1.5 )%  :  MULTIPLIER = 1.5
# [1.5 - 0.5)%  :  MULTIPLIER = 1.0
# [0.5 - 0)%    :  MULTIPLIER = 0.8
# [0 -  0.5)%   :  MULTIPLIER = 0.2
# [0.5  - 1)%   :  MULTIPLIER = 1.0
# [1 -  1.5)%   :  MULTIPLIER = 2.0
# [1.5 -  2)%   :  MULTIPLIER = 3.0
# [ >=   2 )%   :  MULTIPLIER = 4.0
# [ may force it to sell ] ?
        # if self.actions[action] == "LONG":
        #     if self.position == -1:
        #         for b in self.boughts:
        #             self.budget += -b
        #             self.reward += -(b + 1)
                
        #         realized_pnl = self.reward
        #         if self.cumulative_reward:
        #             self.reward = self.reward / max(1, len(self.boughts))
                
        #         # if got bankrupt
        #         if self.sudden_death * len(self.boughts) > self.reward:
        #             self.nextDay = True

        #         self.boughts = []
        #         self.position = 0
        #     elif self.budget > 0:
        #         self.boughts.append(1.0)
        #         self.position = 1
        #         self.budget -= 1.
        
        # elif self.actions[action] == "SHORT":
        #     if self.position == 1:
        #         for b in self.boughts:
        #             self.budget += b
        #             self.reward += b - 1
                
        #         realized_pnl = self.reward
        #         if self.cumulative_reward:
        #             self.reward = self.reward / max(1, len(self.boughts))

        #         if self.sudden_death * len(self.boughts) > self.reward:
        #             self.nextDay = True

        #         self.boughts = []
        #         self.position = 0
        #     elif self.budget > 0:
        #         self.boughts.append(-1.0)
        #         self.position = -1
        #         self.budget -= 1.
        
        # else:
        #     temp = temp * self.position
        #     self.reward = self._get_holding_mutiplier(cum_reward, temp) * temp

        # self.currentTargetIndex += 1
    # def nextDay(self):
    #     self.currentTargetIndex = self.scope - 1
    #     self.currentDay = self.attack_sequence[self.day_index]
    #     self.nextday = False
    #     self.boughts = []
    #     self.reward = 0.
    #     self.position = 0
    #     self.budget = self.budget*0 + 8.        # automation required # meaning power to buy x shares at current price
    #     self._defineState()

    #     return self.state

        # tmpState['code'] = self.targetCode
        # tmpState['day'] = self.currentDay
        # tmpState['index'] = self.currentTargetIndex

In [ ]:
# parameters
# epsilon = .5  # exploration
# large_batches = True  # train in large batches for less I/O overhead.
# min_epsilon = 0.1
# batch_size = 16384 if large_batches else 64
# discount = 0.9    # increasing future discount incentivise future P
# epoch = 100
# win_cnt = 0
# train_cycle = 20000 if large_batches else 30
# conter = 0

# for e in range(epoch):
#   loss = 0.
#   day = 0
#   game_over = False
#   delay = True
#   env.reset()

#   while not game_over:
    
#     input_t = env.nextDay()
#     day_end = False
#     cumReward, realized_pnl, per_success, num_trades = 0, 0, 0, 0
#     if delay and len(exp_replay.memory) > 40000:
#       print('switching training delay off..')
#       delay = False

#     while not day_end:

#       input_tm1 = input_t
#       isRandom = False
#       q = [None]

#       # get next action
#       if np.random.rand() <= epsilon or delay:
#         action = np.random.randint(0, len(env.actions), size=1)[0]
#         isRandom = True
#       else:
#         q = model_interface.predict(input_tm1)
#         action = np.argmax(q[0])

#       if np.nan in q:
#         print("OCCUR NaN!!!", str(counter), str(day), str(input_tm1['code']))
#         # exit()

#       # apply action, get rewards and new state
#       input_t, reward, day_end, game_over, info, pnl, success = env.step(action)
#       per_success += success > 0
#       num_trades += abs(success)
#       cumReward += reward
#       realized_pnl += pnl

#       # store experience
#       exp_replay.remember([input_tm1,
#                 action,
#                 reward,
#                 input_t],
#                 day_end)

#       # adapt model
#       if counter % train_cycle == 0 and not delay:
#         inputs, targets = exp_replay.get_offline_batch(env, model_interface, batch_size)
#         if large_batches:
#           loss = model_interface.fitModel(inputs, targets, 64, 10)
#         else:
#           loss = model_interface.batchTrain(inputs, targets)
#         print('batch = ', str(counter // train_cycle), 'loss = ', str(loss))
      
#       counter += 1

#     day += 1
#     print('day = ', str(day), 'cumReward =', str(cumReward), 'pnl =', str(realized_pnl), 'percentage success', str(float(per_success) * 100 / max(num_trades, 1)))

#     if cumReward > 0:
#       win_cnt += 1
    
#     # kill this game randomly, expected 30 games in a row
#     game_over = np.random.randint(30) == 0
  
#   print('switching game! epsilon=', str(epsilon))
#   # print("Epoch {:03d}/{} | Loss {:.4f} | Win count {} | Epsilon {:.4f}".format(e, epoch, loss, win_cnt, epsilon))
#   # Save trained model weights and architecture, this will be used by the visualization code
#   if day >= 800:
#     epsilon = max(min_epsilon, epsilon * 0.99)
#     model_interface.model.save_weights("model.h5", overwrite=True)
